## Collapse Board Data to School Level, Merge with IPEDS and NACUBO Data

### For Tables 3 and 4 of Elite Embeddedness

By Charlie Eaton and Albina Gibadullina

In [1]:
use unitid firstname middlename lastname suffix undergrad mba jd md phd otherprofdegree ///
alumni schoolname schooltype gender industry_code financetype boardofficer boardchair unitid ///
firmid firmname_new organization forbes forbesyear eigenvector betweenness strength year using d_board_members_all_long20220501, clear

In [2]:
drop if schooltype=="liberal arts"
drop if unitid==232186 | unitid==126818 | unitid==209542 | unitid==221759 | unitid==102614 | unitid==129020 | unitid==181464
drop if year<2003

gen pe=financetype=="Private Equity"
replace pe=1 if financetype=="Venture capital"
gen hedge=financetype=="Hedge funds"
gen nonfinance=financetype=="Non-financial business"
gen otherfinance=0
replace otherfinance=1 if pe!=1 & hedge!=1 & nonfinance!=1


(1,886 observations deleted)

(1,689 observations deleted)

(4,765 observations deleted)


(923 real changes made)




(20,949 real changes made)


In [3]:
collapse (sum) pe hedge otherfinance nonfinance (first) schoolname schooltype , by(unitid year)
gen schooltypenum =2 if schooltype=="public"
replace schooltypenum=3 if schooltype=="research"
label define schooltype 2 "public" 3 "private"
label values schooltypenum schooltype



(440 missing values generated)

(440 real changes made)




In [4]:
merge 1:1 unitid year using d_nacubo_funds_2003_2017, gen(b_merge) keep(1 3)
**correct university of delaware unitid to flagship
replace unitid=130943 if unitid==130343



    Result                      Number of obs
    -----------------------------------------
    Not matched                            22
        from master                        22  (b_merge==1)
        from using                          0  (b_merge==2)

    Matched                               865  (b_merge==3)
    -----------------------------------------

(15 real changes made)


In [5]:
merge 1:1 unitid year using d_f1a_endow_ipeds_2003_2018, nogen keep(1 3)


    Result                      Number of obs
    -----------------------------------------
    Not matched                           471
        from master                       471  
        from using                          0  

    Matched                               416  
    -----------------------------------------


In [6]:
foreach file in board1_forbes_400_count board2_forbes_400_count ///
mba_forbes_400_count ba_forbes_400_count {
 qui   merge 1:1 unitid year using d_`file', gen(m`file') keep(1 3)
}

foreach var in pehedge otherfin technology forbes {
qui    egen `var'board=rowtotal(`var'board1 `var'board2)
}

foreach var in pehedge otherfin technology forbes {
  qui  egen alumni`var'=rowtotal(`var'_ba `var'_mba)
}

In [7]:
cd "/Users/Charlie/Dropbox/Bankers in the Ivory Tower Book/2 Our New Financial Oligarchy/Data/Boards 2000 to 2017"
merge 1:1 unitid year using nonmissing_sat_act_admissions_all_universities_20181223, nogen keep(1 3) keepusing(acceptrate)


/Users/Charlie/Dropbox/Bankers in the Ivory Tower Book/2 Our New Financial Oliga
> rchy/Data/Boards 2000 to 2017


    Result                      Number of obs
    -----------------------------------------
    Not matched                             0
    Matched                               887  
    -----------------------------------------


In [8]:
cd "/Users/Charlie/Google Drive/Lincoln Project/IPEDS Disaggregated/Data"
*merge 1:1 unitid year using hd_flags_1981_2018, keepusing(instnm control fips) keep(1 3)
merge 1:1 unitid year using finaid_2000_2017, keepusing(fgrnt_p) nogen keep(1 3)

merge 1:1 unitid year using fteenrollment1981_to_2018, nogen ///
keepusing(fte_count ftfirsteftotlt ptfirsteftotlt ftfirstefasiat ptfirstefasiat ftfirstefwhitt ptfirstefwhitt) ///
keep(1 3)

**change rutgers unitid for years that it reported revenue at different level
replace unitid=186362 if unitid==186380 & year<2005
merge 1:1 unitid year using financial_all_1981_2017, ///
keepusing(total03_revenue) keep(1 3)

replace unitid=186380 if unitid==186362

merge m:1 year using cpi_scalar2017, nogen keep(1 3)


/Users/Charlie/Google Drive/Lincoln Project/IPEDS Disaggregated/Data


    Result                      Number of obs
    -----------------------------------------
    Not matched                             0
    Matched                               887  
    -----------------------------------------


    Result                      Number of obs
    -----------------------------------------
    Not matched                             0
    Matched                               887  
    -----------------------------------------

(2 real changes made)


    Result                      Number of obs
    -----------------------------------------
    Not matched                             0
    Matched                               887  (_merge==3)
    -----------------------------------------

(2 real changes made)


    Result                      Number of obs
    -----------------------------------------
    Not matched                             0
    Matched                    

In [9]:
rename EndowmentFunds endowment_assets
replace endowment_assets=endowment_public_ipeds if endowment_assets==.



(17 real changes made)


In [10]:
cd "/Users/Charlie/Dropbox/Stata_Notebooks/highfinancesocialcircuitry/data"

/Users/Charlie/Dropbox/Stata_Notebooks/highfinancesocialcircuitry/data


In [11]:
ds, varwidth(28)

unitid                    mboard1_forbes_400_count  technologyboard
year                      forbesboard2              forbesboard
pe                        pehedgeboard2             alumnipehedge
hedge                     otherfinboard2            alumniotherfin
otherfinance              technologyboard2          alumnitechnology
nonfinance                mboard2_forbes_400_count  alumniforbes
schoolname                forbes_mba                acceptrate
schooltype                pehedge_mba               fgrnt_p
schooltypenum             otherfin_mba              ftfirsteftotlt
endowment_assets          technology_mba            ptfirsteftotlt
State                     mmba_forbes_400_count     ftfirstefasiat
Institution               forbes_ba                 ptfirstefasiat
b_merge                   pehedge_ba                ftfirstefwhitt
endowment_public_ipeds    otherfin_ba               ptfirstefwhitt
forbesboard1              technology_ba             fte_count
pehedgeboard1 

In [12]:
save d_school_level_elite_embed, replace

file d_school_level_elite_embed.dta saved
